In [1]:
import numpy as np
import pandas as pd
import os
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import numpy as np
import math
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

In [2]:
import pickle
from torch.utils.data import Dataset
from pathlib import Path

# Load N-Cars dataset
DATASET_PATH = Path("ncars_dataset.pkl")

print("Loading N-Cars dataset...")
with open(DATASET_PATH, "rb") as f:
    ncars_data = pickle.load(f)

print(f"Train sequences: {len(ncars_data['train'])}")
print(f"Test sequences: {len(ncars_data['test'])}")

Loading N-Cars dataset...
Train sequences: 15423
Test sequences: 8606


In [3]:
# Create PyTorch Dataset for N-Cars
class NCarsDatasetForTTFS(Dataset):
    def __init__(self, sequences, num_time_bins=10, height=100, width=120):
        self.sequences = sequences
        self.num_time_bins = num_time_bins
        self.height = height
        self.width = width
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        seq = self.sequences[idx]
        xs, ys, ts, ps = seq['x'], seq['y'], seq['t'], seq['p']
        label = seq['label']
        
        # Normalize timestamps
        if len(ts) > 0:
            t_min, t_max = ts.min(), ts.max()
            ts_norm = (ts - t_min) / (t_max - t_min) if t_max > t_min else np.zeros_like(ts)
        else:
            ts_norm = np.array([])
        
        # Create tensor in format [C=2, X=120, Y=100, T=10]
        ttfs_tensor = np.zeros((2, self.width, self.height, self.num_time_bins), dtype=np.float32)
        
        # Fill events
        if len(xs) > 0:
            valid = (xs >= 0) & (xs < self.width) & (ys >= 0) & (ys < self.height)
            xs_v, ys_v, ts_v, ps_v = xs[valid], ys[valid], ts_norm[valid], ps[valid]
            
            t_bins = np.clip((ts_v * (self.num_time_bins - 1)).astype(np.int32), 0, self.num_time_bins - 1)
            spike_vals = np.ones_like(ts_v)  # Unit spike magnitude
            
            for x, y, t_bin, p, val in zip(xs_v, ys_v, t_bins, ps_v, spike_vals):
                pol_ch = 1 if p == 1 else 0
                ttfs_tensor[pol_ch, x, y, t_bin] += val
                # Add temporal persistence
                for future_bin in range(t_bin + 1, min(t_bin + 3, self.num_time_bins)):
                    decay = 0.5 ** (future_bin - t_bin)
                    ttfs_tensor[pol_ch, x, y, future_bin] += val * decay
        
        ttfs_tensor = np.clip(ttfs_tensor, 0, 10.0)
        return torch.from_numpy(ttfs_tensor), label

# Create datasets
train_dataset = NCarsDatasetForTTFS(ncars_data['train'], num_time_bins=10)
test_dataset = NCarsDatasetForTTFS(ncars_data['test'], num_time_bins=10)

print(f"\nTrain dataset: {len(train_dataset)} samples")
print(f"Test dataset: {len(test_dataset)} samples")


Train dataset: 15423 samples
Test dataset: 8606 samples


In [4]:
# Check data format
sample_data, sample_label = train_dataset[0]
print(f"Sample data shape: {sample_data.shape}")  # Should be [2, 120, 100, 10]
print(f"Sample label: {sample_label}")
print(f"Data range: [{sample_data.min():.2f}, {sample_data.max():.2f}]")

Sample data shape: torch.Size([2, 120, 100, 10])
Sample label: 1
Data range: [0.00, 7.00]


In [5]:
from torch.utils.data import DataLoader, random_split

# # Split your dataset
# train_size = int(0.95 * len(train_dataset))
# val_size = len(train_dataset) - train_size
# train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [8]:
import torch
import torch.nn as nn
from spikingjelly.clock_driven import functional
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import time
import math
import os

class TTFSNeuron(nn.Module):
    """
    Time-to-First-Spike (TTFS) neuron implementation based on IBM's identity mapping approach
    Uses the B1-model (constant slope B=1) for equivalent training with ReLU networks
    """
    def __init__(self, shape, tau_c=1.0, v_threshold=1.0, v_reset=0.0, spike_count=True):
        super().__init__()
        self.shape = shape if isinstance(shape, (list, tuple)) else [shape]
        self.tau_c = tau_c
        self.v_threshold = v_threshold
        self.v_reset = v_reset
        self.spike_count = spike_count
        
        # B1-model parameters (identity mapping)
        self.A = 0.0  # Initial slope = 0 
        self.B = 1.0  # Fixed slope = 1 (identity mapping condition)
        
        # Time parameters (will be set dynamically)
        self.t_min = 0.0
        self.t_max = 10.0
        
        # For fully connected layers, create scalar threshold adjustment
        if isinstance(shape, int):
            self.threshold_adjustment = nn.Parameter(torch.zeros(shape))
            self.num_neurons = shape
        else:
            # For conv layers, create per-channel threshold adjustment
            self.threshold_adjustment = nn.Parameter(torch.zeros(shape[0]))
            self.num_neurons = shape[0] if len(shape) > 0 else 1
        
        # Spike counting statistics
        self.reset_spike_stats()
        
    def reset_spike_stats(self):
        """Reset spike counting statistics"""
        self.total_spikes = 0
        self.total_timesteps = 0
        self.spikes_per_neuron = []
        self.current_timestep_spikes = 0
        
    def reset_state(self):
        """Reset neuron state"""
        pass  # TTFS neurons are stateless between time steps
    
    def forward(self, x):
        """
        Forward pass for TTFS neuron - simplified to work like ReLU with learnable threshold
        This maintains the mathematical equivalence while being trainable
        """
        batch_size = x.shape[0]
        
        # Apply learnable threshold adjustment
        if len(x.shape) == 4:  # Conv layer [N, C, H, W]
            # Reshape threshold for broadcasting: [C] -> [1, C, 1, 1]
            threshold = self.threshold_adjustment.view(1, -1, 1, 1)
            spatial_neurons = x.shape[2] * x.shape[3]  # H * W
        elif len(x.shape) == 2:  # FC layer [N, features]
            # Use threshold directly: [features]
            threshold = self.threshold_adjustment
            spatial_neurons = 1
        else:
            threshold = self.threshold_adjustment
            spatial_neurons = 1
        
        # For identity mapping (B1-model), the equivalent ReLU operation is:
        # output = max(0, input - threshold_adjustment)
        # This is mathematically equivalent to the TTFS spike time conversion
        output = torch.relu(x - threshold)
        
        # Calculate spikes for this timestep
        spiking_mask = output > 0
        
        if self.spike_count:
            # Count spikes per neuron in this timestep
            if len(x.shape) == 4:  # Conv layer
                # Sum spikes across spatial dimensions for each channel
                spikes_per_channel = spiking_mask.sum(dim=(0, 2, 3)).float()
                total_neurons_per_channel = batch_size * spatial_neurons
                spikes_per_neuron = spikes_per_channel / total_neurons_per_channel
            else:  # FC layer
                # Sum spikes across batch dimension
                spikes_per_feature = spiking_mask.sum(dim=0).float()
                spikes_per_neuron = spikes_per_feature / batch_size
            
            self.spikes_per_neuron.append(spikes_per_neuron.detach().cpu())
            self.current_timestep_spikes = spiking_mask.sum().item()
            self.total_spikes += self.current_timestep_spikes
            self.total_timesteps += 1
        
        # For compatibility, return format expected by calling code
        spike_times = self.t_max - self.tau_c * output  # Reverse conversion for logging
        
        return output, spike_times, spiking_mask
    
    def get_spike_statistics(self):
        """Get spike statistics for this neuron layer"""
        if not self.spikes_per_neuron:
            return {
                'avg_spikes_per_neuron': 0.0,
                'total_spikes': 0,
                'total_timesteps': 0,
                'spike_rate': 0.0
            }
        
        # Average spikes per neuron across all timesteps
        all_spikes = torch.stack(self.spikes_per_neuron)  # [timesteps, neurons]
        avg_spikes_per_neuron = all_spikes.mean().item()
        
        return {
            'avg_spikes_per_neuron': avg_spikes_per_neuron,
            'total_spikes': self.total_spikes,
            'total_timesteps': self.total_timesteps,
            'spike_rate': self.total_spikes / max(self.total_timesteps, 1),
            'spikes_per_timestep': all_spikes.mean(dim=1).tolist()  # Average spikes per timestep
        }

class TTFSConvNeuron(TTFSNeuron):
    """TTFS neuron for convolutional layers"""
    def __init__(self, channels, height=None, width=None, **kwargs):
        # For conv layers, we only need the number of channels for threshold adjustment
        # The spatial dimensions (height, width) are handled automatically by broadcasting
        super().__init__([channels], **kwargs)

class SpikingCNN_NCARS_TTFS(nn.Module):
    """
    TTFS-based Spiking CNN implementing IBM's equivalent training method with majority voting
    Uses identity mapping (B1-model) for exact equivalence with ReLU networks
    
    Modified for data shape: [N, C=2, X=120, Y=100, T=10]
    Now predicts for each timestep and uses majority voting for final classification
    Enhanced with an additional initial convolutional layer
    """
    def __init__(self, tau_c=1.0, v_threshold=1.0, v_reset=0.0, num_classes=2, 
                 voting_method='majority', confidence_weighted=False):
        super().__init__()
        
        self.tau_c = tau_c
        self.v_threshold = v_threshold
        self.v_reset = v_reset
        self.num_classes = num_classes
        self.voting_method = voting_method  # 'majority', 'average', 'weighted'
        self.confidence_weighted = confidence_weighted
        
        # Time parameters for each layer (will be adaptive) - added one more for new layer
        self.t_max = [0.0, 5.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0]
        self.t_min = [0.0, 5.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0]
        
        # New initial convolutional block - input has 2 channels
        self.conv0 = nn.Conv2d(2, 32, kernel_size=5, stride=1, padding=2, bias=False)
        self.bn0 = nn.BatchNorm2d(32)
        self.ttfs0 = TTFSConvNeuron(32, 120, 100, tau_c=tau_c, v_threshold=v_threshold)
        self.pool0 = nn.AvgPool2d(kernel_size=2, stride=2)  # Average pooling for TTFS
        
        # First convolutional block - now takes 32 channels from conv0
        self.conv1 = nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.ttfs1 = TTFSConvNeuron(32, 60, 50, tau_c=tau_c, v_threshold=v_threshold)
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)  # Average pooling for TTFS
        
        # Second convolutional block
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2, bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        self.ttfs2 = TTFSConvNeuron(64, 30, 25, tau_c=tau_c, v_threshold=v_threshold)
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        # Third convolutional block
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128)
        self.ttfs3 = TTFSConvNeuron(128, 15, 12, tau_c=tau_c, v_threshold=v_threshold)
        self.pool3 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        # Fully connected layers - adjusted for new spatial dimensions after 4 pooling layers
        # After 4 pooling layers (2x each): 120 -> 60 -> 30 -> 15 -> 7 (with rounding)
        # And: 100 -> 50 -> 25 -> 12 -> 6 (with rounding)
        self.fc1 = nn.Linear(128 * 7 * 6, 256, bias=False)
        self.ttfs4 = TTFSNeuron(256, tau_c=tau_c, v_threshold=v_threshold)
        
        self.fc2 = nn.Linear(256, 64, bias=False)
        self.ttfs5 = TTFSNeuron(64, tau_c=tau_c, v_threshold=v_threshold)
        
        # Output layer (non-spiking readout)
        self.fc3 = nn.Linear(64, num_classes, bias=False)
        
        # Optional confidence network for weighted voting
        if confidence_weighted:
            self.confidence_net = nn.Sequential(
                nn.Linear(64, 32),
                nn.ReLU(),
                nn.Linear(32, 1),
                nn.Sigmoid()
            )
        
        # Initialize weights using method compatible with identity mapping
        self._initialize_weights()
        
        # Layer names for spike statistics - added ttfs0
        self.spike_layers = ['ttfs0', 'ttfs1', 'ttfs2', 'ttfs3', 'ttfs4', 'ttfs5']
    
    def _initialize_weights(self):
        """
        Initialize weights for stable TTFS training
        Based on IBM's identity mapping approach
        """
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                # Use He initialization for ReLU-like behavior
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def reset_spike_statistics(self):
        """Reset spike statistics for all TTFS layers"""
        for layer_name in self.spike_layers:
            layer = getattr(self, layer_name)
            layer.reset_spike_stats()
    
    def get_network_spike_statistics(self):
        """Get spike statistics for the entire network"""
        stats = {}
        total_network_spikes = 0
        total_network_neurons = 0
        
        for layer_name in self.spike_layers:
            layer = getattr(self, layer_name)
            layer_stats = layer.get_spike_statistics()
            stats[layer_name] = layer_stats
            
            # Accumulate network totals
            total_network_spikes += layer_stats['total_spikes']
            total_network_neurons += layer.num_neurons
        
        # Calculate network-wide averages
        stats['network_summary'] = {
            'total_spikes': total_network_spikes,
            'total_neurons': total_network_neurons,
            'avg_spikes_per_neuron': total_network_spikes / max(total_network_neurons, 1),
            'layers': len(self.spike_layers)
        }
        
        return stats
    
    def update_time_windows(self, layer_outputs):
        """
        Adaptive time window update to ensure spikes occur within bounds
        """
        gamma = 0.1  # Safety margin
        
        for i, output in enumerate(layer_outputs):
            if output is not None and len(output) > 1:  # Has spike times
                spike_times = output[1]  # Get spike times
                if spike_times.numel() > 0:
                    # Filter spike times within current window
                    valid_spikes = spike_times[spike_times < self.t_max[i]]
                    if valid_spikes.numel() > 0:  # Check if any valid spikes exist
                        max_spike_time = torch.max(valid_spikes)
                        if max_spike_time.item() > 0:
                            # Update t_max with safety margin
                            new_t_max = max_spike_time.item() + gamma * (self.t_max[i] - self.t_min[i])
                            self.t_max[i] = min(new_t_max, self.t_max[i])
                            
                            # Update subsequent time windows
                            if i + 1 < len(self.t_min):
                                self.t_min[i + 1] = self.t_max[i]
    
    def reset_neurons(self):
        """Reset all TTFS neurons"""
        self.ttfs0.reset_state()
        self.ttfs1.reset_state()
        self.ttfs2.reset_state()
        self.ttfs3.reset_state()
        self.ttfs4.reset_state()
        self.ttfs5.reset_state()
    
    def apply_majority_vote(self, timestep_outputs, timestep_confidences=None):
        """
        Apply majority voting across timestep predictions
        
        Args:
            timestep_outputs: List of [batch_size, num_classes] tensors, one per timestep
            timestep_confidences: Optional list of confidence scores per timestep
            
        Returns:
            final_output: [batch_size, num_classes] final prediction
            voting_info: Dictionary with voting statistics
        """
        if not timestep_outputs:
            raise ValueError("No timestep outputs provided for voting")
        
        batch_size = timestep_outputs[0].shape[0]
        num_timesteps = len(timestep_outputs)
        
        if self.voting_method == 'majority':
            # Hard majority voting
            timestep_predictions = torch.stack([torch.argmax(output, dim=1) for output in timestep_outputs], dim=1)
            
            # Count votes for each class
            vote_counts = torch.zeros(batch_size, self.num_classes, device=timestep_predictions.device)
            for i in range(batch_size):
                for t in range(num_timesteps):
                    pred_class = timestep_predictions[i, t]
                    vote_counts[i, pred_class] += 1
            
            # Final prediction is the class with most votes
            final_predictions = torch.argmax(vote_counts, dim=1)
            
            # Create one-hot encoded output for compatibility with loss functions
            final_output = torch.zeros(batch_size, self.num_classes, device=timestep_predictions.device)
            final_output.scatter_(1, final_predictions.unsqueeze(1), 1.0)
            
            # Convert to logit-like scores (add small epsilon to avoid log(0))
            final_output = torch.log(final_output + 1e-8)
            
            voting_info = {
                'timestep_predictions': timestep_predictions,
                'vote_counts': vote_counts,
                'final_predictions': final_predictions,
                'consensus_strength': vote_counts.max(dim=1)[0] / num_timesteps  # Fraction of votes for winner
            }
            
        elif self.voting_method == 'average':
            # Soft voting - average the logits/outputs
            stacked_outputs = torch.stack(timestep_outputs, dim=1)  # [batch, timesteps, classes]
            
            if timestep_confidences is not None:
                # Confidence-weighted average
                confidences = torch.stack(timestep_confidences, dim=1)  # [batch, timesteps, 1]
                weights = F.softmax(confidences, dim=1)
                final_output = (stacked_outputs * weights).sum(dim=1)
            else:
                # Simple average
                final_output = stacked_outputs.mean(dim=1)
            
            final_predictions = torch.argmax(final_output, dim=1)
            
            voting_info = {
                'timestep_outputs': stacked_outputs,
                'final_predictions': final_predictions,
                'weights': weights if timestep_confidences is not None else None,
                'consensus_strength': F.softmax(final_output, dim=1).max(dim=1)[0]  # Max probability
            }
            
        elif self.voting_method == 'weighted':
            # Weighted voting based on prediction confidence
            stacked_outputs = torch.stack(timestep_outputs, dim=1)  # [batch, timesteps, classes]
            
            # Use max probability as confidence measure
            timestep_probs = F.softmax(stacked_outputs, dim=2)
            timestep_confidences = timestep_probs.max(dim=2)[0]  # [batch, timesteps]
            
            # Normalize confidences to create weights
            weights = F.softmax(timestep_confidences, dim=1).unsqueeze(2)  # [batch, timesteps, 1]
            
            # Weighted average
            final_output = (stacked_outputs * weights).sum(dim=1)
            final_predictions = torch.argmax(final_output, dim=1)
            
            voting_info = {
                'timestep_outputs': stacked_outputs,
                'timestep_confidences': timestep_confidences,
                'weights': weights.squeeze(2),
                'final_predictions': final_predictions,
                'consensus_strength': F.softmax(final_output, dim=1).max(dim=1)[0]
            }
        
        else:
            raise ValueError(f"Unknown voting method: {self.voting_method}")
        
        return final_output, voting_info
    
    def forward(self, x, return_timestep_outputs=False):
        """
        Forward pass processing each timestep sequentially
        Input shape: [N, C=2, X=120, Y=100, T=10]
        """
        # Data should already be in shape [N, C=2, X=120, Y=100, T=10]
        # If your data comes in a different format, adjust here:
        # Example: if data is [N, T, C, X, Y], permute to [N, C, X, Y, T]
        # x = x.permute(0, 2, 3, 4, 1)
        
        N, C, X, Y, T = x.shape
        
        # Verify expected input format
        assert C == 2, f"Expected 2 channels, got {C}"
        assert X == 120, f"Expected width 120, got {X}"
        assert Y == 100, f"Expected height 100, got {Y}"
        assert T == 10, f"Expected 10 timesteps, got {T}"
        
        # Process each time step and collect outputs
        timestep_outputs = []
        timestep_confidences = [] if self.confidence_weighted else None
        
        # Reset spike statistics at the beginning of forward pass
        if self.training:
            self.reset_spike_statistics()
        
        for t in range(T):
            # Extract frame at time t: [N, C, X, Y]
            x_t = x[:, :, :, :, t]
            
            # New initial conv block
            x_t = self.conv0(x_t)
            x_t = self.bn0(x_t)
            x_t, spike_times0, mask0 = self.ttfs0(x_t)
            x_t = self.pool0(x_t)
            
            # First conv block
            x_t = self.conv1(x_t)
            x_t = self.bn1(x_t)
            x_t, spike_times1, mask1 = self.ttfs1(x_t)
            x_t = self.pool1(x_t)
            
            # Second conv block  
            x_t = self.conv2(x_t)
            x_t = self.bn2(x_t)
            x_t, spike_times2, mask2 = self.ttfs2(x_t)
            x_t = self.pool2(x_t)
            
            # Third conv block
            x_t = self.conv3(x_t)
            x_t = self.bn3(x_t)
            x_t, spike_times3, mask3 = self.ttfs3(x_t)
            x_t = self.pool3(x_t)
            
            # Flatten
            x_t = x_t.reshape(N, -1)
            
            # Fully connected layers
            x_t = self.fc1(x_t)
            x_t, spike_times4, mask4 = self.ttfs4(x_t)
            
            x_t = self.fc2(x_t)
            x_t, spike_times5, mask5 = self.ttfs5(x_t)
            
            # Output layer (non-spiking)
            output_t = self.fc3(x_t)
            timestep_outputs.append(output_t)
            
            # Optional confidence prediction
            if self.confidence_weighted:
                confidence_t = self.confidence_net(x_t)
                timestep_confidences.append(confidence_t)
        
        # Apply majority voting across timesteps
        final_output, voting_info = self.apply_majority_vote(timestep_outputs, timestep_confidences)
        
        if return_timestep_outputs:
            return final_output, timestep_outputs, voting_info
        else:
            return final_output

def train_ncars_ttfs_snn_majority(train_loader, val_loader=None, num_epochs=20, log_spikes=True, 
                                 save_best=True, save_all_epochs=True, 
                                 model_save_dir='models', model_name_prefix='ttfs_snn_majority',
                                 voting_method='majority', confidence_weighted=False,
                                 timestep_loss_weight=0.1):
    """
    Train TTFS SNN with majority voting using both final and timestep losses
    Modified to match Code 2's data handling and display format
    
    Args:
        timestep_loss_weight: Weight for auxiliary timestep losses (0.0 to disable)
        voting_method: 'majority', 'average', or 'weighted'
        confidence_weighted: Whether to use confidence weighting
    """
    # Create save directory if it doesn't exist
    if save_all_epochs or save_best:
        os.makedirs(model_save_dir, exist_ok=True)
        print(f"📁 Model save directory: {model_save_dir}")
    
    # Hyperparameters
    learning_rate = 1e-3
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🚀 Starting TTFS SNN training with {voting_method} voting on device: {device}")
    print(f"📊 Training samples: {len(train_loader.dataset)}")
    if val_loader:
        print(f"📊 Validation samples: {len(val_loader.dataset)}")
    print(f"🎯 Number of epochs: {num_epochs}")
    print(f"📡 Data format: [N, C=2, X=120, Y=100, T=10]")
    print(f"🗳️  Voting method: {voting_method}")
    print(f"⚖️  Timestep loss weight: {timestep_loss_weight}")
    print(f"🧠 Architecture: 4 conv layers + 2 FC layers (with new initial conv layer)")
    if save_best:
        print(f"💾 Best model will be saved to: {model_save_dir}")
    if save_all_epochs:
        print(f"💾 All epoch models will be saved to: {model_save_dir}")
    print("=" * 80)
    
    # Model, loss, optimizer
    model = SpikingCNN_NCARS_TTFS(tau_c=1.0, v_threshold=1.0, v_reset=0.0, num_classes=2,
                                 voting_method=voting_method, confidence_weighted=confidence_weighted).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    
    # Print model info
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"🧠 TTFS SNN parameters: {total_params:,}")
    print(f"🎯 Trainable parameters: {trainable_params:,}")
    print(f"⚡ Using Time-to-First-Spike coding with {voting_method} voting")
    print(f"🔬 Method: IBM equivalent training (Nature Communications 2024)")
    print("=" * 80)
    
    # Training history
    train_losses = []
    train_accuracies = []
    val_accuracies = []
    spike_statistics_history = []
    voting_statistics_history = []
    
    # Best model tracking
    best_accuracy = 0.0
    best_epoch = 0
    best_model_state = None
    
    # Training loop
    model.train()
    start_time = time.time()
    
    for epoch in range(num_epochs):
        epoch_start = time.time()
        total_loss = 0
        total_final_loss = 0
        total_timestep_loss = 0
        correct = 0
        total = 0
        
        # Progress bar for batches
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', 
                   unit='batch', leave=False)
        
        # Reset statistics
        epoch_spike_stats = []
        epoch_voting_stats = []
        
        for batch_idx, (data, target) in enumerate(pbar):
            data, target = data.to(device), target.to(device)
            
            # Data should already be in shape [N, C=2, X=120, Y=100, T=10]
            # If your data comes in a different format, adjust here:
            # Example: if data is [N, T, C, X, Y], permute to [N, C, X, Y, T]
            # data = data.permute(0, 2, 3, 4, 1)
            
            # Forward pass with timestep outputs
            final_output, timestep_outputs, voting_info = model(data, return_timestep_outputs=True)
            
            # Main loss on final majority vote output
            final_loss = criterion(final_output, target)
            
            # Optional auxiliary loss on individual timestep predictions
            timestep_loss = 0.0
            if timestep_loss_weight > 0.0:
                for t_output in timestep_outputs:
                    timestep_loss += criterion(t_output, target)
                timestep_loss = timestep_loss / len(timestep_outputs)
            
            # Combined loss
            loss = final_loss + timestep_loss_weight * timestep_loss
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            
            # Gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            # Statistics
            total_loss += loss.item()
            total_final_loss += final_loss.item()
            total_timestep_loss += timestep_loss if isinstance(timestep_loss, (int, float)) else timestep_loss.item()
            
            # Accuracy calculation
            if voting_method == 'majority':
                predicted = voting_info['final_predictions']
            else:
                _, predicted = final_output.max(1)
            
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            
            # Log statistics
            if log_spikes and batch_idx % 10 == 0:
                batch_spike_stats = model.get_network_spike_statistics()
                epoch_spike_stats.append(batch_spike_stats)
                
                # Voting statistics
                consensus = voting_info['consensus_strength'].mean().item()
                epoch_voting_stats.append({
                    'consensus_strength': consensus,
                    'voting_method': voting_method
                })
            
            # Update progress bar
            current_acc = 100. * correct / total
            pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'FinalL': f'{final_loss.item():.4f}',
                'TsL': f'{timestep_loss:.4f}' if isinstance(timestep_loss, (int, float)) else f'{timestep_loss.item():.4f}',
                'Acc': f'{current_acc:.2f}%'
            })
        
        pbar.close()
        
        # Step learning rate scheduler
        scheduler.step()
        
        # Epoch statistics
        avg_loss = total_loss / len(train_loader)
        avg_final_loss = total_final_loss / len(train_loader)
        avg_timestep_loss = total_timestep_loss / len(train_loader)
        train_acc = 100. * correct / total
        epoch_time = time.time() - epoch_start
        
        train_losses.append(avg_loss)
        train_accuracies.append(train_acc)
        
        # Statistics logging
        if epoch_spike_stats:
            avg_spikes_per_neuron = np.mean([stats['network_summary']['avg_spikes_per_neuron'] 
                                           for stats in epoch_spike_stats])
            avg_consensus = np.mean([stats['consensus_strength'] for stats in epoch_voting_stats])
            
            spike_statistics_history.append({
                'epoch': epoch + 1,
                'avg_spikes_per_neuron': avg_spikes_per_neuron,
                'detailed_stats': epoch_spike_stats[-1]
            })
            
            voting_statistics_history.append({
                'epoch': epoch + 1,
                'avg_consensus_strength': avg_consensus,
                'voting_method': voting_method
            })
        
        print(f"📈 Epoch {epoch+1:2d} | "
              f"Loss: {avg_loss:.4f} (Final: {avg_final_loss:.4f}, TS: {avg_timestep_loss:.4f}) | "
              f"Train Acc: {train_acc:6.2f}% | "
              f"Time: {epoch_time:5.1f}s | "
              f"LR: {scheduler.get_last_lr()[0]:.2e}")
        
        # Print additional statistics
        if log_spikes and epoch_spike_stats:
            avg_spikes = np.mean([stats['network_summary']['avg_spikes_per_neuron'] 
                                for stats in epoch_spike_stats])
            avg_consensus = np.mean([stats['consensus_strength'] for stats in epoch_voting_stats])
            print(f"⚡ Avg spikes/neuron: {avg_spikes:.3f} | Voting consensus: {avg_consensus:.3f}")
            
            # Print detailed layer statistics
            last_stats = epoch_spike_stats[-1]
            for layer_name in ['ttfs0', 'ttfs1', 'ttfs2', 'ttfs3', 'ttfs4', 'ttfs5']:
                layer_spikes = last_stats[layer_name]['avg_spikes_per_neuron']
                print(f"   {layer_name}: {layer_spikes:.3f} spikes/neuron")
        
        # Validation
        current_accuracy = train_acc  # Default to training accuracy
        if val_loader is not None:
            print("🔍 Running validation...", end=" ")
            val_acc = test_ncars_ttfs_snn_majority(model, val_loader, device, verbose=False)
            val_accuracies.append(val_acc)
            current_accuracy = val_acc
            print(f"Val Acc: {val_acc:6.2f}%")
            model.train()
        
        # Create epoch model metadata
        epoch_metadata = {
            'epoch': epoch + 1,
            'train_accuracy': train_acc,
            'val_accuracy': val_acc if val_loader else None,
            'train_loss': avg_loss,
            'final_loss': avg_final_loss,
            'timestep_loss': avg_timestep_loss,
            'voting_method': voting_method,
            'confidence_weighted': confidence_weighted,
            'timestep_loss_weight': timestep_loss_weight,
            'learning_rate': scheduler.get_last_lr()[0],
            'spike_statistics': epoch_spike_stats[-1] if epoch_spike_stats else None,
            'voting_statistics': epoch_voting_stats[-1] if epoch_voting_stats else None,
            'optimizer_state': optimizer.state_dict(),
            'scheduler_state': scheduler.state_dict(),
            'total_epochs': num_epochs,
            'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
        }
        
        # Save models
        if save_all_epochs:
            epoch_model_path = os.path.join(model_save_dir, f"{model_name_prefix}_epoch_{epoch+1:02d}.pth")
            save_ttfs_model_state(model.state_dict(), epoch_model_path, epoch_metadata)
            print(f"💾 Epoch {epoch+1} model saved: {epoch_model_path}")
        
        # Save best model
        if save_best and current_accuracy > best_accuracy:
            best_accuracy = current_accuracy
            best_epoch = epoch + 1
            best_model_state = model.state_dict().copy()
            
            best_metadata = epoch_metadata.copy()
            best_metadata.update({
                'best_accuracy': best_accuracy,
                'best_epoch': best_epoch,
                'model_type': 'best_model',
            })
            
            best_model_path = os.path.join(model_save_dir, f"{model_name_prefix}_best.pth")
            save_ttfs_model_state(best_model_state, best_model_path, best_metadata)
            print(f"🏆 New best model saved! Accuracy: {best_accuracy:.2f}% (Epoch {best_epoch})")
        
        print("-" * 80)
    
    # Save final model
    if save_all_epochs:
        final_model_path = os.path.join(model_save_dir, f"{model_name_prefix}_final.pth")
        final_metadata = epoch_metadata.copy()
        final_metadata.update({
            'model_type': 'final_model',
            'final_train_accuracy': train_acc,
            'final_val_accuracy': val_acc if val_loader else None,
        })
        save_ttfs_model_state(model.state_dict(), final_model_path, final_metadata)
        print(f"🔚 Final model saved: {final_model_path}")
    
    # Training summary
    total_time = time.time() - start_time
    print("🎉 TTFS SNN majority voting training completed!")
    print(f"⏱️  Total training time: {total_time/60:.1f} minutes")
    print(f"📈 Best training accuracy: {max(train_accuracies):.2f}%")
    if val_accuracies:
        print(f"📊 Best validation accuracy: {max(val_accuracies):.2f}%")
    
    # Print saved model information
    print(f"\n💾 Saved Models Summary:")
    if save_all_epochs:
        print(f"   📁 Epoch models: {model_save_dir}/{model_name_prefix}_epoch_XX.pth")
        print(f"   📁 Final model: {model_save_dir}/{model_name_prefix}_final.pth")
    if save_best:
        print(f"   🏆 Best model: {model_save_dir}/{model_name_prefix}_best.pth")
        print(f"       Best accuracy: {best_accuracy:.2f}% at epoch {best_epoch}")
    
    # Final spike statistics summary
    if spike_statistics_history:
        final_spikes = spike_statistics_history[-1]['avg_spikes_per_neuron']
        print(f"⚡ Final average spikes per neuron: {final_spikes:.3f}")
    
    return {
        'train_losses': train_losses,
        'train_accuracies': train_accuracies,
        'val_accuracies': val_accuracies if val_accuracies else None,
        'spike_statistics': spike_statistics_history,
        'voting_statistics': voting_statistics_history,
        'best_epoch': best_epoch,
        'best_accuracy': best_accuracy,
        'voting_method': voting_method,
        'model_save_dir': model_save_dir,
        'saved_models': {
            'best_model': f"{model_save_dir}/{model_name_prefix}_best.pth" if save_best else None,
            'final_model': f"{model_save_dir}/{model_name_prefix}_final.pth" if save_all_epochs else None,
            'epoch_models': [f"{model_save_dir}/{model_name_prefix}_epoch_{i+1:02d}.pth" 
                           for i in range(num_epochs)] if save_all_epochs else None,
        }
    }

def test_ncars_ttfs_snn_majority(model, test_loader, device, verbose=True, log_spikes=True, analyze_voting=True):
    """Test the trained TTFS SNN with majority voting and detailed analysis"""
    model.eval()
    test_loss = 0
    correct = 0
    criterion = nn.CrossEntropyLoss()
    
    if verbose:
        print("🧪 Testing TTFS SNN model with majority voting...")
        pbar = tqdm(test_loader, desc='Testing', unit='batch')
    else:
        pbar = test_loader
    
    test_spike_stats = []
    voting_analysis = {
        'consensus_scores': [],
        'timestep_accuracies': [[] for _ in range(10)],  # Assuming T=10
        'final_accuracies': [],
        'disagreement_cases': []
    }
    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(pbar):
            data, target = data.to(device), target.to(device)
            
            # Forward pass with timestep outputs
            final_output, timestep_outputs, voting_info = model(data, return_timestep_outputs=True)
            
            # Compute loss
            test_loss += criterion(final_output, target).item()
            
            # Accuracy calculation
            if model.voting_method == 'majority':
                predicted = voting_info['final_predictions']
            else:
                _, predicted = final_output.max(1)
            
            correct += predicted.eq(target).sum().item()
            
            # Detailed voting analysis
            if analyze_voting:
                consensus = voting_info['consensus_strength']
                voting_analysis['consensus_scores'].extend(consensus.cpu().tolist())
                
                # Analyze individual timestep accuracies
                for t, t_output in enumerate(timestep_outputs):
                    _, t_pred = t_output.max(1)
                    t_acc = t_pred.eq(target).float()
                    voting_analysis['timestep_accuracies'][t].extend(t_acc.cpu().tolist())
                
                # Final accuracy for this batch
                final_acc = predicted.eq(target).float()
                voting_analysis['final_accuracies'].extend(final_acc.cpu().tolist())
                
                # Find disagreement cases (where timesteps disagree significantly)
                if model.voting_method == 'majority' and 'timestep_predictions' in voting_info:
                    timestep_preds = voting_info['timestep_predictions']
                    for i in range(timestep_preds.shape[0]):
                        sample_preds = timestep_preds[i]  # [T]
                        unique_preds = torch.unique(sample_preds)
                        if len(unique_preds) > 1:  # Disagreement exists
                            disagreement_ratio = 1.0 - (sample_preds == sample_preds[0]).float().mean().item()
                            if disagreement_ratio > 0.3:  # Significant disagreement
                                voting_analysis['disagreement_cases'].append({
                                    'sample_idx': batch_idx * data.shape[0] + i,
                                    'timestep_predictions': sample_preds.cpu().tolist(),
                                    'final_prediction': predicted[i].item(),
                                    'true_label': target[i].item(),
                                    'disagreement_ratio': disagreement_ratio,
                                    'consensus_strength': consensus[i].item()
                                })
            
            # Log spike statistics
            if log_spikes and batch_idx % 10 == 0:
                batch_spike_stats = model.get_network_spike_statistics()
                test_spike_stats.append(batch_spike_stats)
            
            if verbose and hasattr(pbar, 'set_postfix'):
                current_acc = 100. * correct / ((batch_idx + 1) * data.shape[0])
                pbar.set_postfix({'Acc': f'{current_acc:.2f}%'})
    
    if verbose and hasattr(pbar, 'close'):
        pbar.close()
    
    test_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)
    
    if verbose:
        print(f"✅ TTFS SNN Test Results with {model.voting_method} voting:")
        print(f"   Loss: {test_loss:.4f}")
        print(f"   Accuracy: {accuracy:.2f}%")
        
        # Voting analysis results
        if analyze_voting:
            avg_consensus = np.mean(voting_analysis['consensus_scores'])
            print(f"   Average consensus strength: {avg_consensus:.3f}")
            
            # Timestep accuracy analysis
            print("   Per-timestep accuracies:")
            for t in range(len(voting_analysis['timestep_accuracies'])):
                if voting_analysis['timestep_accuracies'][t]:
                    t_acc = np.mean(voting_analysis['timestep_accuracies'][t]) * 100
                    print(f"     Timestep {t+1}: {t_acc:.2f}%")
            
            # Disagreement analysis
            if voting_analysis['disagreement_cases']:
                print(f"   High disagreement cases: {len(voting_analysis['disagreement_cases'])}")
                print(f"   Average disagreement ratio: {np.mean([case['disagreement_ratio'] for case in voting_analysis['disagreement_cases']]):.3f}")
        
        # Spike statistics
        if test_spike_stats:
            avg_spikes = np.mean([stats['network_summary']['avg_spikes_per_neuron'] 
                                for stats in test_spike_stats])
            print(f"   Average spikes per neuron: {avg_spikes:.3f}")
            
            # Detailed layer statistics
            last_stats = test_spike_stats[-1]
            print("📊 Per-layer spike statistics:")
            for layer_name in ['ttfs0', 'ttfs1', 'ttfs2', 'ttfs3', 'ttfs4', 'ttfs5']:
                layer_spikes = last_stats[layer_name]['avg_spikes_per_neuron']
                print(f"   {layer_name}: {layer_spikes:.3f} spikes/neuron")
    
    test_results = {
        'accuracy': accuracy,
        'loss': test_loss,
        'voting_analysis': voting_analysis if analyze_voting else None,
        'spike_statistics': test_spike_stats
    }
    
    return accuracy if not verbose else test_results

def compare_voting_methods(model_state_dict, test_loader, device='cpu', num_classes=2):
    """Compare different voting methods using the same trained model weights"""
    print("🔍 Comparing voting methods with same trained weights:")
    print("=" * 60)
    
    voting_methods = ['majority', 'average', 'weighted']
    results = {}
    
    for method in voting_methods:
        print(f"\n🗳️  Testing with {method} voting:")
        
        # Create model with specific voting method
        model = SpikingCNN_NCARS_TTFS(num_classes=num_classes, voting_method=method).to(device)
        model.load_state_dict(model_state_dict)
        
        # Test the model
        test_results = test_ncars_ttfs_snn_majority(model, test_loader, device, 
                                                   verbose=True, log_spikes=False, analyze_voting=True)
        
        results[method] = test_results
        print(f"   {method.title()} voting accuracy: {test_results['accuracy']:.2f}%")
        if test_results['voting_analysis']:
            avg_consensus = np.mean(test_results['voting_analysis']['consensus_scores'])
            print(f"   Average consensus: {avg_consensus:.3f}")
    
    # Summary comparison
    print(f"\n📊 Voting Method Comparison Summary:")
    print("-" * 40)
    for method in voting_methods:
        acc = results[method]['accuracy']
        consensus = np.mean(results[method]['voting_analysis']['consensus_scores']) if results[method]['voting_analysis'] else 0
        print(f"{method.title():<10}: {acc:6.2f}% (consensus: {consensus:.3f})")
    
    # Find best method
    best_method = max(results.keys(), key=lambda x: results[x]['accuracy'])
    print(f"\n🏆 Best voting method: {best_method.title()} ({results[best_method]['accuracy']:.2f}%)")
    
    return results

def analyze_temporal_patterns(model, test_loader, device, num_samples=100):
    """Analyze how predictions change across timesteps"""
    print("\n📊 Analyzing temporal prediction patterns...")
    
    model.eval()
    temporal_analysis = {
        'timestep_confidences': [[] for _ in range(10)],
        'prediction_stability': [],
        'early_vs_late_accuracy': {'early': [], 'late': []},
        'convergence_patterns': []
    }
    
    sample_count = 0
    with torch.no_grad():
        for data, target in test_loader:
            if sample_count >= num_samples:
                break
                
            data, target = data.to(device), target.to(device)
            
            # Get timestep outputs
            final_output, timestep_outputs, voting_info = model(data, return_timestep_outputs=True)
            
            batch_size = data.shape[0]
            for i in range(min(batch_size, num_samples - sample_count)):
                sample_timestep_outputs = [t_out[i:i+1] for t_out in timestep_outputs]
                
                # Analyze confidence evolution
                confidences = [F.softmax(t_out, dim=1).max().item() for t_out in sample_timestep_outputs]
                for t, conf in enumerate(confidences):
                    temporal_analysis['timestep_confidences'][t].append(conf)
                
                # Prediction stability (how much predictions change across time)
                predictions = [t_out.argmax().item() for t_out in sample_timestep_outputs]
                stability = len(set(predictions)) / len(predictions)  # Lower = more stable
                temporal_analysis['prediction_stability'].append(1.0 - stability)
                
                # Early vs late accuracy
                true_label = target[i].item()
                early_preds = predictions[:3]  # First 3 timesteps
                late_preds = predictions[-3:]  # Last 3 timesteps
                
                early_correct = sum(1 for p in early_preds if p == true_label) / len(early_preds)
                late_correct = sum(1 for p in late_preds if p == true_label) / len(late_preds)
                
                temporal_analysis['early_vs_late_accuracy']['early'].append(early_correct)
                temporal_analysis['early_vs_late_accuracy']['late'].append(late_correct)
                
                # Convergence pattern
                final_pred = predictions[-1]
                convergence_timestep = -1
                for t in range(len(predictions)):
                    if all(p == final_pred for p in predictions[t:]):
                        convergence_timestep = t
                        break
                temporal_analysis['convergence_patterns'].append(convergence_timestep)
                
                sample_count += 1
                if sample_count >= num_samples:
                    break
    
    # Print analysis results
    print(f"\n📈 Temporal Pattern Analysis Results:")
    print(f"Average prediction stability: {np.mean(temporal_analysis['prediction_stability']):.3f}")
    
    early_acc = np.mean(temporal_analysis['early_vs_late_accuracy']['early'])
    late_acc = np.mean(temporal_analysis['early_vs_late_accuracy']['late'])
    print(f"Early accuracy (first 3 steps): {early_acc*100:.2f}%")
    print(f"Late accuracy (last 3 steps): {late_acc*100:.2f}%")
    print(f"Improvement over time: {(late_acc - early_acc)*100:.2f}%")
    
    # Confidence evolution
    print(f"Confidence evolution across timesteps:")
    for t in range(10):
        if temporal_analysis['timestep_confidences'][t]:
            avg_conf = np.mean(temporal_analysis['timestep_confidences'][t])
            print(f"   Timestep {t+1}: {avg_conf:.3f}")
    
    # Convergence analysis
    valid_convergence = [c for c in temporal_analysis['convergence_patterns'] if c >= 0]
    if valid_convergence:
        avg_convergence = np.mean(valid_convergence)
        print(f"Average convergence timestep: {avg_convergence:.1f}")
        print(f"Samples with convergence: {len(valid_convergence)}/{len(temporal_analysis['convergence_patterns'])} ({len(valid_convergence)/len(temporal_analysis['convergence_patterns'])*100:.1f}%)")
    
    return temporal_analysis

def save_ttfs_model_state(model_state_dict, filepath, metadata=None):
    """Save TTFS model state dict with metadata"""
    save_dict = {
        'model_state_dict': model_state_dict,
        'model_class': 'SpikingCNN_NCARS_TTFS',
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
        'method': 'IBM TTFS with Identity Mapping and Majority Voting',
        'paper': 'High-performance deep spiking neural networks with 0.3 spikes per neuron (Nature Communications 2024)',
        'data_format': '[N, C=2, X=120, Y=100, T=10]',
        'architecture': '4 conv layers (with initial conv0) + 2 FC layers',
    }
    if metadata:
        save_dict.update(metadata)
    
    torch.save(save_dict, filepath)

def load_ttfs_model_majority(filepath, device='cpu', voting_method='majority', confidence_weighted=False):
    """Load saved TTFS model with majority voting"""
    checkpoint = torch.load(filepath, map_location=device)
    
    # Use voting method from checkpoint if available, otherwise use parameter
    saved_voting_method = checkpoint.get('voting_method', voting_method)
    saved_confidence_weighted = checkpoint.get('confidence_weighted', confidence_weighted)
    
    model = SpikingCNN_NCARS_TTFS(voting_method=saved_voting_method, 
                                 confidence_weighted=saved_confidence_weighted)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    
    print(f"📂 TTFS model loaded from: {filepath}")
    print(f"🗳️  Voting method: {saved_voting_method}")
    if saved_confidence_weighted:
        print("⚖️  Confidence weighting: enabled")
    
    if 'model_type' in checkpoint:
        if checkpoint['model_type'] == 'best_model':
            print(f"🏆 This is a best model checkpoint")
            if 'best_accuracy' in checkpoint:
                print(f"   🎯 Best accuracy: {checkpoint['best_accuracy']:.2f}%")
            if 'best_epoch' in checkpoint:
                print(f"   📍 Best epoch: {checkpoint['best_epoch']}")
        elif checkpoint['model_type'] == 'final_model':
            print(f"🔚 This is a final model checkpoint")
            if 'final_train_accuracy' in checkpoint:
                print(f"   🎯 Final train accuracy: {checkpoint['final_train_accuracy']:.2f}%")
            if 'final_val_accuracy' in checkpoint and checkpoint['final_val_accuracy']:
                print(f"   📊 Final val accuracy: {checkpoint['final_val_accuracy']:.2f}%")
        else:
            print(f"📅 This is an epoch {checkpoint.get('epoch', 'unknown')} checkpoint")
    
    if 'epoch' in checkpoint:
        print(f"   📍 Epoch: {checkpoint['epoch']}")
    if 'train_accuracy' in checkpoint:
        print(f"   🎯 Train accuracy: {checkpoint['train_accuracy']:.2f}%")
    if 'val_accuracy' in checkpoint and checkpoint['val_accuracy']:
        print(f"   📊 Val accuracy: {checkpoint['val_accuracy']:.2f}%")
    if 'voting_statistics' in checkpoint and checkpoint['voting_statistics']:
        consensus = checkpoint['voting_statistics'].get('avg_consensus_strength', 'N/A')
        print(f"   🗳️  Average consensus: {consensus}")
    if 'spike_statistics' in checkpoint and checkpoint['spike_statistics']:
        stats = checkpoint['spike_statistics']
        if 'network_summary' in stats:
            avg_spikes = stats['network_summary']['avg_spikes_per_neuron']
            print(f"   ⚡ Spikes per neuron: {avg_spikes:.3f}")
    
    return model, checkpoint

def list_saved_models(model_save_dir, model_name_prefix='ttfs_snn_majority'):
    """List all saved models in the directory"""
    if not os.path.exists(model_save_dir):
        print(f"❌ Directory {model_save_dir} does not exist")
        return []
    
    import glob
    pattern = os.path.join(model_save_dir, f"{model_name_prefix}*.pth")
    model_files = glob.glob(pattern)
    
    if not model_files:
        print(f"❌ No models found with prefix '{model_name_prefix}' in {model_save_dir}")
        return []
    
    print(f"📁 Found {len(model_files)} saved models in {model_save_dir}:")
    
    # Sort models by type
    best_models = [f for f in model_files if 'best' in f]
    final_models = [f for f in model_files if 'final' in f]
    epoch_models = [f for f in model_files if 'epoch' in f and 'best' not in f and 'final' not in f]
    
    if best_models:
        print("🏆 Best models:")
        for model_file in sorted(best_models):
            print(f"   {os.path.basename(model_file)}")
    
    if final_models:
        print("🔚 Final models:")
        for model_file in sorted(final_models):
            print(f"   {os.path.basename(model_file)}")
    
    if epoch_models:
        print("📅 Epoch models:")
        for model_file in sorted(epoch_models):
            print(f"   {os.path.basename(model_file)}")
    
    return sorted(model_files)

def compare_models(model_files, test_loader, device='cpu'):
    """Compare multiple saved models on test set"""
    print(f"\n🔍 Comparing {len(model_files)} models:")
    print("=" * 60)
    
    results = []
    
    for model_file in model_files:
        print(f"\n📂 Testing: {os.path.basename(model_file)}")
        try:
            model, checkpoint = load_ttfs_model_majority(model_file, device)
            accuracy = test_ncars_ttfs_snn_majority(model, test_loader, device, verbose=False, log_spikes=False)
            
            result = {
                'filename': os.path.basename(model_file),
                'filepath': model_file,
                'accuracy': accuracy,
                'epoch': checkpoint.get('epoch', 'unknown'),
                'train_accuracy': checkpoint.get('train_accuracy', 'unknown'),
                'val_accuracy': checkpoint.get('val_accuracy', 'unknown'),
                'model_type': checkpoint.get('model_type', 'epoch_model'),
                'voting_method': checkpoint.get('voting_method', 'unknown')
            }
            results.append(result)
            
            print(f"   ✅ Test accuracy: {accuracy:.2f}%")
            
        except Exception as e:
            print(f"   ❌ Error loading model: {str(e)}")
    
    # Sort by accuracy and display summary
    if results:
        results.sort(key=lambda x: x['accuracy'], reverse=True)
        
        print(f"\n📊 Model Comparison Summary:")
        print("=" * 100)
        print(f"{'Rank':<4} {'Model':<30} {'Voting':<8} {'Epoch':<6} {'Test Acc':<9} {'Train Acc':<10} {'Val Acc':<9}")
        print("-" * 100)
        
        for i, result in enumerate(results, 1):
            epoch_str = str(result['epoch']) if result['epoch'] != 'unknown' else 'N/A'
            train_str = f"{result['train_accuracy']:.2f}%" if result['train_accuracy'] != 'unknown' else 'N/A'
            val_str = f"{result['val_accuracy']:.2f}%" if result['val_accuracy'] != 'unknown' and result['val_accuracy'] is not None else 'N/A'
            voting_str = result['voting_method'] if result['voting_method'] != 'unknown' else 'N/A'
            
            print(f"{i:<4} {result['filename']:<30} {voting_str:<8} {epoch_str:<6} {result['accuracy']:<9.2f}% {train_str:<10} {val_str:<9}")
        
        print("=" * 100)
        print(f"🏆 Best performing model: {results[0]['filename']} ({results[0]['accuracy']:.2f}%)")
    
    return results

def save_training_history(history, filepath):
    """Save complete training history including spike statistics"""
    save_dict = {
        'training_history': history,
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
        'data_format': '[N, C=2, X=120, Y=100, T=10]',
    }
    
    torch.save(save_dict, filepath)
    print(f"📊 Training history saved to: {filepath}")

def load_training_history(filepath):
    """Load training history"""
    checkpoint = torch.load(filepath, map_location='cpu')
    print(f"📈 Training history loaded from: {filepath}")
    return checkpoint['training_history']

def analyze_spike_patterns(spike_statistics_history):
    """Analyze spike patterns across training epochs"""
    print("\n📊 Spike Pattern Analysis:")
    print("=" * 50)
    
    if not spike_statistics_history:
        print("No spike statistics available.")
        return
    
    epochs = [stat['epoch'] for stat in spike_statistics_history]
    avg_spikes = [stat['avg_spikes_per_neuron'] for stat in spike_statistics_history]
    
    print(f"📈 Spike rate evolution:")
    print(f"   Initial: {avg_spikes[0]:.3f} spikes/neuron")
    print(f"   Final: {avg_spikes[-1]:.3f} spikes/neuron")
    print(f"   Min: {min(avg_spikes):.3f} spikes/neuron")
    print(f"   Max: {max(avg_spikes):.3f} spikes/neuron")
    
    # Analyze per-layer patterns from the last epoch
    if spike_statistics_history:
        last_detailed = spike_statistics_history[-1]['detailed_stats']
        print(f"\n🔍 Final per-layer analysis:")
        layer_names = ['ttfs0', 'ttfs1', 'ttfs2', 'ttfs3', 'ttfs4', 'ttfs5']
        layer_types = ['Conv0', 'Conv1', 'Conv2', 'Conv3', 'FC1', 'FC2']
        
        for layer_name, layer_type in zip(layer_names, layer_types):
            if layer_name in last_detailed:
                stats = last_detailed[layer_name]
                print(f"   {layer_type:5} ({layer_name}): {stats['avg_spikes_per_neuron']:.3f} spikes/neuron")
                if 'spikes_per_timestep' in stats and len(stats['spikes_per_timestep']) > 0:
                    temporal_pattern = stats['spikes_per_timestep']
                    print(f"         Temporal: {[f'{x:.2f}' for x in temporal_pattern[:5]]}...")
    
    # Check if within IBM target range
    target_range = 0.3
    final_rate = avg_spikes[-1]
    if final_rate <= target_range:
        print(f"\n✅ Target achieved: {final_rate:.3f} ≤ {target_range} spikes/neuron")
    else:
        print(f"\n⚠️  Above target: {final_rate:.3f} > {target_range} spikes/neuron")
        print("   Consider adjusting threshold parameters")

def benchmark_spike_efficiency(model, data_loader, device, num_batches=5):
    """Benchmark spike efficiency of the TTFS network"""
    print("\n⚡ Spike Efficiency Benchmark:")
    print("=" * 40)
    
    model.eval()
    total_spikes = 0
    total_neurons = 0
    total_timesteps = 0
    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            if batch_idx >= num_batches:
                break
                
            data = data.to(device)
            
            # Reset spike statistics
            model.reset_spike_statistics()
            
            # Forward pass
            _ = model(data)
            
            # Collect spike statistics
            stats = model.get_network_spike_statistics()
            
            batch_spikes = stats['network_summary']['total_spikes']
            batch_neurons = stats['network_summary']['total_neurons']
            
            total_spikes += batch_spikes
            total_neurons += batch_neurons
            total_timesteps += data.shape[-1]  # T dimension
            
            print(f"   Batch {batch_idx + 1}: {batch_spikes / batch_neurons:.3f} spikes/neuron")
    
    avg_spike_rate = total_spikes / total_neurons if total_neurons > 0 else 0
    energy_efficiency = 1.0 / (avg_spike_rate + 1e-8)  # Inverse of spike rate
    
    print(f"\n📊 Efficiency Summary:")
    print(f"   Average spike rate: {avg_spike_rate:.3f} spikes/neuron")
    print(f"   Energy efficiency: {energy_efficiency:.1f}x")
    print(f"   Total spikes: {total_spikes:,}")
    print(f"   Total neurons: {total_neurons:,}")
    
    # Compare with traditional ANN (theoretical)
    ann_activations_per_neuron = 1.0  # Each neuron activates once per timestep
    efficiency_gain = ann_activations_per_neuron / avg_spike_rate
    print(f"   Efficiency vs ANN: {efficiency_gain:.1f}x reduction in operations")

def create_data_example():
    """Create example data in the expected format [N, C=2, X=120, Y=100, T=10]"""
    print("📋 Creating example data in format [N, C=2, X=120, Y=100, T=10]:")
    
    # Create synthetic data
    batch_size = 8
    channels = 2
    height = 120
    width = 100
    timesteps = 10
    
    # Generate synthetic spatio-temporal data
    data = torch.randn(batch_size, channels, height, width, timesteps)
    
    # Create synthetic labels (binary classification)
    labels = torch.randint(0, 2, (batch_size,))
    
    print(f"   Data shape: {data.shape}")
    print(f"   Labels shape: {labels.shape}")
    print(f"   Data type: {data.dtype}")
    print(f"   Labels type: {labels.dtype}")
    
    return data, labels

# Example usage
def main_majority_voting():
    print("🧠 TTFS Spiking CNN with Majority Vote Classification (Enhanced)")
    print("=" * 70)
    print("📄 Implementation of 'High-performance deep spiking neural networks")
    print("   with 0.3 spikes per neuron' (Nature Communications 2024)")
    print("=" * 70)
    print("🎯 Enhanced version with:")
    print("   • Additional initial convolutional layer (conv0)")
    print("   • Timestep-wise predictions with majority voting")
    print("   • Multiple voting strategies (majority, average, weighted)")
    print("   • Auxiliary timestep loss for better predictions")
    print("   • Detailed voting and spike analysis")
    print("   • Temporal pattern analysis")
    print("   • Model comparison utilities")
    print("   • Epoch-wise model saving")
    print("=" * 70)
    print("📡 Data format: [N, C=2, X=120, Y=100, T=10]")
    print("   N: Batch size")
    print("   C: 2 channels (e.g., polarity events)")
    print("   X: 120 pixels width")
    print("   Y: 100 pixels height")
    print("   T: 10 temporal frames")
    print("=" * 70)
    print("🏗️  Architecture: 4 conv layers + 2 FC layers")
    print("   • Conv0: 2 → 32 channels (NEW)")
    print("   • Conv1: 32 → 32 channels")
    print("   • Conv2: 32 → 64 channels")
    print("   • Conv3: 64 → 128 channels")
    print("   • FC1: 128×7×6 → 256")
    print("   • FC2: 256 → 64")
    print("   • Output: 64 → 2 classes")
    print("=" * 70)
    
    # Create example data
    example_data, example_labels = create_data_example()
    
    print("\n📝 Usage Example:")
    print("""
from torch.utils.data import DataLoader, TensorDataset

# Create your dataset with shape [N, C=2, X=120, Y=100, T=10]
dataset = TensorDataset(data_tensor, labels_tensor)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# Train with different voting methods
voting_methods = ['majority', 'average', 'weighted']

for method in voting_methods:
    print(f"Training with {method} voting...")
    
    history = train_ncars_ttfs_snn_majority(
        train_loader, 
        val_loader,
        num_epochs=20,
        voting_method=method,
        timestep_loss_weight=0.1,  # Auxiliary loss weight
        save_best=True,
        save_all_epochs=True,
        model_name_prefix=f'ttfs_snn_{method}'
    )

# List all saved models
model_files = list_saved_models('models', 'ttfs_snn_majority')

# Compare all saved models
results = compare_models(model_files, test_loader, device)

# Load and test specific model
model, checkpoint = load_ttfs_model_majority('models/ttfs_snn_majority_best.pth', device)
test_results = test_ncars_ttfs_snn_majority(model, test_loader, device)

# Compare voting methods with same weights
best_model_path = 'models/ttfs_snn_majority_best.pth'
checkpoint = torch.load(best_model_path)
comparison = compare_voting_methods(checkpoint['model_state_dict'], test_loader, device)

# Analyze temporal patterns
model, _ = load_ttfs_model_majority(best_model_path)
temporal_analysis = analyze_temporal_patterns(model, test_loader, device)

# Analyze spike patterns
analyze_spike_patterns(history['spike_statistics'])

# Benchmark efficiency
benchmark_spike_efficiency(model, test_loader, device)

# Save training history
save_training_history(history, 'training_history.pth')
""")
    
    print("\n🎯 Key Features:")
    print("   • Additional initial conv layer for better feature extraction")
    print("   • Timestep-wise predictions with final majority voting")
    print("   • Multiple voting strategies: majority, average, weighted")
    print("   • Auxiliary timestep loss for better individual predictions")
    print("   • Detailed voting analysis and consensus tracking")
    print("   • Temporal pattern analysis across timesteps")
    print("   • Comparison utilities for different voting methods")
    print("   • Early vs late prediction accuracy analysis")
    print("   • Prediction stability and convergence metrics")
    print("   • Epoch-wise model saving with complete metadata")
    print("   • Best model tracking and automatic saving")
    print("   • Spike efficiency benchmarking")
    print("   • Enhanced depth with 4 convolutional layers")

if __name__ == "__main__":
    main_majority_voting()

🧠 TTFS Spiking CNN with Majority Vote Classification (Enhanced)
📄 Implementation of 'High-performance deep spiking neural networks
   with 0.3 spikes per neuron' (Nature Communications 2024)
🎯 Enhanced version with:
   • Additional initial convolutional layer (conv0)
   • Timestep-wise predictions with majority voting
   • Multiple voting strategies (majority, average, weighted)
   • Auxiliary timestep loss for better predictions
   • Detailed voting and spike analysis
   • Temporal pattern analysis
   • Model comparison utilities
   • Epoch-wise model saving
📡 Data format: [N, C=2, X=120, Y=100, T=10]
   N: Batch size
   C: 2 channels (e.g., polarity events)
   X: 120 pixels width
   Y: 100 pixels height
   T: 10 temporal frames
🏗️  Architecture: 4 conv layers + 2 FC layers
   • Conv0: 2 → 32 channels (NEW)
   • Conv1: 32 → 32 channels
   • Conv2: 32 → 64 channels
   • Conv3: 64 → 128 channels
   • FC1: 128×7×6 → 256
   • FC2: 256 → 64
   • Output: 64 → 2 classes
📋 Creating example da

In [9]:
method = 'majority'

history = train_ncars_ttfs_snn_majority(
        train_loader, 
        val_loader,
        num_epochs=20,
        voting_method=method,
        timestep_loss_weight=0.1,  # Auxiliary loss weight
        save_best=True,
        save_all_epochs=True,
        model_name_prefix=f'ttfs_snn_{method}'
    )

📁 Model save directory: models
🚀 Starting TTFS SNN training with majority voting on device: cuda
📊 Training samples: 15423
📊 Validation samples: 8606
🎯 Number of epochs: 20
📡 Data format: [N, C=2, X=120, Y=100, T=10]
🗳️  Voting method: majority
⚖️  Timestep loss weight: 0.1
🧠 Architecture: 4 conv layers + 2 FC layers (with new initial conv layer)
💾 Best model will be saved to: models
💾 All epoch models will be saved to: models
🧠 TTFS SNN parameters: 1,545,984
🎯 Trainable parameters: 1,545,984
⚡ Using Time-to-First-Spike coding with majority voting
🔬 Method: IBM equivalent training (Nature Communications 2024)


📈 Epoch  1 | Loss: 1.6955 (Final: 1.6709, TS: 0.2458) | Train Acc:  90.93% | Time: 1027.6s | LR: 1.00e-03
⚡ Avg spikes/neuron: 191841.585 | Voting consensus: 0.962
   ttfs0: 0.716 spikes/neuron
   ttfs1: 0.624 spikes/neuron
   ttfs2: 0.491 spikes/neuron
   ttfs3: 0.500 spikes/neuron
   ttfs4: 0.147 spikes/neuron
   ttfs5: 0.323 spikes/neuron
🔍 Running validation... Val Acc:  83.52%
💾 Epoch 1 model saved: models\ttfs_snn_majority_epoch_01.pth
🏆 New best model saved! Accuracy: 83.52% (Epoch 1)
--------------------------------------------------------------------------------


📈 Epoch  2 | Loss: 0.6509 (Final: 0.6389, TS: 0.1194) | Train Acc:  96.53% | Time: 1034.8s | LR: 1.00e-03
⚡ Avg spikes/neuron: 207819.118 | Voting consensus: 0.980
   ttfs0: 0.751 spikes/neuron
   ttfs1: 0.654 spikes/neuron
   ttfs2: 0.476 spikes/neuron
   ttfs3: 0.419 spikes/neuron
   ttfs4: 0.129 spikes/neuron
   ttfs5: 0.324 spikes/neuron
🔍 Running validation... Val Acc:  90.00%
💾 Epoch 2 model saved: models\ttfs_snn_majority_epoch_02.pth
🏆 New best model saved! Accuracy: 90.00% (Epoch 2)
--------------------------------------------------------------------------------


📈 Epoch  3 | Loss: 0.5102 (Final: 0.5004, TS: 0.0975) | Train Acc:  97.28% | Time: 1028.1s | LR: 1.00e-03
⚡ Avg spikes/neuron: 208078.854 | Voting consensus: 0.982
   ttfs0: 0.734 spikes/neuron
   ttfs1: 0.614 spikes/neuron
   ttfs2: 0.439 spikes/neuron
   ttfs3: 0.415 spikes/neuron
   ttfs4: 0.110 spikes/neuron
   ttfs5: 0.325 spikes/neuron
🔍 Running validation... Val Acc:  90.91%
💾 Epoch 3 model saved: models\ttfs_snn_majority_epoch_03.pth
🏆 New best model saved! Accuracy: 90.91% (Epoch 3)
--------------------------------------------------------------------------------


📈 Epoch  4 | Loss: 0.4750 (Final: 0.4658, TS: 0.0921) | Train Acc:  97.47% | Time: 1043.7s | LR: 1.00e-03
⚡ Avg spikes/neuron: 208006.638 | Voting consensus: 0.985
   ttfs0: 0.772 spikes/neuron
   ttfs1: 0.631 spikes/neuron
   ttfs2: 0.406 spikes/neuron
   ttfs3: 0.330 spikes/neuron
   ttfs4: 0.125 spikes/neuron
   ttfs5: 0.322 spikes/neuron
🔍 Running validation... Val Acc:  91.44%
💾 Epoch 4 model saved: models\ttfs_snn_majority_epoch_04.pth
🏆 New best model saved! Accuracy: 91.44% (Epoch 4)
--------------------------------------------------------------------------------


...

📈 Epoch 16 | Loss: 0.0947 (Final: 0.0920, TS: 0.0277) | Train Acc:  99.50% | Time: 969.6s | LR: 5.00e-04
⚡ Avg spikes/neuron: 183772.289 | Voting consensus: 0.993
   ttfs0: 0.743 spikes/neuron
   ttfs1: 0.350 spikes/neuron
   ttfs2: 0.173 spikes/neuron
   ttfs3: 0.113 spikes/neuron
   ttfs4: 0.083 spikes/neuron
   ttfs5: 0.317 spikes/neuron
🔍 Running validation... Val Acc:  93.62%
💾 Epoch 16 model saved: models\ttfs_snn_majority_epoch_16.pth
🏆 New best model saved! Accuracy: 93.62% (Epoch 16)
--------------------------------------------------------------------------------


📈 Epoch 17 | Loss: 0.0848 (Final: 0.0824, TS: 0.0244) | Train Acc:  99.55% | Time: 965.9s | LR: 5.00e-04
⚡ Avg spikes/neuron: 182858.075 | Voting consensus: 0.993
   ttfs0: 0.759 spikes/neuron
   ttfs1: 0.327 spikes/neuron
   ttfs2: 0.136 spikes/neuron
   ttfs3: 0.083 spikes/neuron
   ttfs4: 0.111 spikes/neuron
   ttfs5: 0.293 spikes/neuron
🔍 Running validation... Val Acc:  93.82%
💾 Epoch 17 model saved: models\ttfs_snn_majority_epoch_17.pth
🏆 New best model saved! Accuracy: 93.82% (Epoch 17)
--------------------------------------------------------------------------------


📈 Epoch 18 | Loss: 0.0946 (Final: 0.0920, TS: 0.0264) | Train Acc:  99.50% | Time: 970.8s | LR: 5.00e-04
⚡ Avg spikes/neuron: 182735.985 | Voting consensus: 0.994
   ttfs0: 0.742 spikes/neuron
   ttfs1: 0.342 spikes/neuron
   ttfs2: 0.135 spikes/neuron
   ttfs3: 0.101 spikes/neuron
   ttfs4: 0.096 spikes/neuron
   ttfs5: 0.293 spikes/neuron
🔍 Running validation... Val Acc:  90.84%
💾 Epoch 18 model saved: models\ttfs_snn_majority_epoch_18.pth
--------------------------------------------------------------------------------


📈 Epoch 19 | Loss: 0.0679 (Final: 0.0657, TS: 0.0221) | Train Acc:  99.64% | Time: 960.2s | LR: 5.00e-04
⚡ Avg spikes/neuron: 175261.817 | Voting consensus: 0.991
   ttfs0: 0.718 spikes/neuron
   ttfs1: 0.289 spikes/neuron
   ttfs2: 0.138 spikes/neuron
   ttfs3: 0.082 spikes/neuron
   ttfs4: 0.077 spikes/neuron
   ttfs5: 0.326 spikes/neuron
🔍 Running validation... Val Acc:  93.12%
💾 Epoch 19 model saved: models\ttfs_snn_majority_epoch_19.pth
--------------------------------------------------------------------------------


📈 Epoch 20 | Loss: 0.0715 (Final: 0.0693, TS: 0.0223) | Train Acc:  99.62% | Time: 870.4s | LR: 2.50e-04
⚡ Avg spikes/neuron: 174101.982 | Voting consensus: 0.995
   ttfs0: 0.720 spikes/neuron
   ttfs1: 0.298 spikes/neuron
   ttfs2: 0.122 spikes/neuron
   ttfs3: 0.078 spikes/neuron
   ttfs4: 0.072 spikes/neuron
   ttfs5: 0.399 spikes/neuron
🔍 Running validation... Val Acc:  90.50%
💾 Epoch 20 model saved: models\ttfs_snn_majority_epoch_20.pth
--------------------------------------------------------------------------------
🔚 Final model saved: models\ttfs_snn_majority_final.pth
🎉 TTFS SNN majority voting training completed!
⏱️  Total training time: 503.9 minutes
📈 Best training accuracy: 99.64%
📊 Best validation accuracy: 93.82%

💾 Saved Models Summary:
   📁 Epoch models: models/ttfs_snn_majority_epoch_XX.pth
   📁 Final model: models/ttfs_snn_majority_final.pth
   🏆 Best model: models/ttfs_snn_majority_best.pth
       Best accuracy: 93.82% at epoch 17
⚡ Final average spikes per neuron: 1

In [25]:
save_training_history(history, 'training_history.pth')

📊 Training history saved to: training_history.pth
